<a href="https://colab.research.google.com/github/preetamjumech/LLM/blob/main/Rag_Ollama3_Chromadb_22_10_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install ollama streamlit chromadb pypdf

In [ ]:
import streamlit as st
import ollama
import chromadb
import pypdf
import os


chromadb.api.client.SharedSystemClient.clear_system_cache()
# Initialize ChromaDB client
chroma_client = chromadb.Client()

# Check if the collection exists before deleting
collections = chroma_client.list_collections()  # Get the list of collections
if "RagChromadb" in collections:
    chroma_client.delete_collection(name="RagChromadb")
collection = chroma_client.get_or_create_collection("RagChromadb")

# Set Streamlit page configuration
st.set_page_config(page_title="Document Q&A App", layout="wide")

# Header
st.title("📄 Document Q&A App")

# File upload section
st.header("Upload PDF Document")
uploaded_file = st.file_uploader("Choose a PDF file", type="pdf")

if uploaded_file is not None:
    # Save the uploaded file temporarily
    with open("uploaded_document.pdf", "wb") as f:
        f.write(uploaded_file.getbuffer())
    st.success("✅ Document uploaded!")

    # Load the PDF and add it to the collection
    def upload_pdf(file_path):
        with open(file_path, 'rb') as file:
            pdf_reader = pypdf.PdfReader(file)

            num_pages = len(pdf_reader.pages)  # Get the number of pages
            st.write(f"Number of pages in the document: {num_pages}")

            id = 0
            for page in pdf_reader.pages:
                collection.add(
                    documents=[page.extract_text()],
                    ids=[f"{file_path}_{id}"]
                )
                id += 1

    # Process the uploaded PDF
    upload_pdf("uploaded_document.pdf")
    st.success("✨ Embeddings created!")

# Query section
st.header("❓ Ask a Question")
query = st.text_input("Enter your question here:")

if st.button("Get Answer") and query:
    similar_docs = collection.query(
        query_texts=[query],
        n_results=3
    )

    if similar_docs["documents"]:
        num_results = len(similar_docs["documents"][0])
        st.write(f"Found {num_results} similar documents.")

        # Prepare messages for the chat model
        messages = []
        for i in range(min(num_results, 3)):
            messages.append({"role": "system", "content": similar_docs["documents"][0][i]})

        messages.append({"role": "user", "content": query})

        # Get response from the chat model
        response = ollama.chat(
            model="llama3:instruct",
            messages=messages
        )

        st.write("📝 Answer:", response["message"]["content"])
    else:
        st.write("No similar documents found.")
